Assume you're given a table Twitter tweet data, write a query to obtain a histogram of tweets posted per user in 2022. Output the tweet count per user as the bucket and the number of Twitter users who fall into that bucket.

In [ ]:
SELECT tweet_count_per_user AS tweet_bucket, COUNT(user_id) users_num
FROM
  (SELECT user_id, COUNT(*) tweet_count_per_user
  FROM tweets
  WHERE to_char(tweet_date::DATE, 'YYYY') = '2022'
  GROUP BY user_id) tweet_per_user
GROUP BY 1


SELECT 
  tweet_count_per_user AS tweet_bucket, 
  COUNT(user_id) AS users_num 
FROM (
  SELECT 
    user_id, 
    COUNT(tweet_id) AS tweet_count_per_user 
  FROM tweets 
  WHERE tweet_date BETWEEN '2022-01-01' 
    AND '2022-12-31'
  GROUP BY user_id) AS total_tweets 
GROUP BY tweet_count_per_user

Given a table of candidates and their skills, you're tasked with finding the candidates best suited for an open Data Science job. You want to find candidates who are proficient in Python, Tableau, and PostgreSQL.

Write a query to list the candidates who possess all of the required skills for the job. Sort the output by candidate ID in ascending order.

Assumption:

There are no duplicates in the candidates table:

| Column Name  | Type    |
| :----------- | :------ |
| candidate_id | integer |
| skill        | varchar |

Example Input:

| candidate_id | skill      |
| :----------- | :--------- |
| 123          | Python     |
| 123          | Tableau    |
| 123          | PostgreSQL |
| 234          | R          |
| 234          | PowerBI    |
| 234          | SQL Server |
| 345          | Python     |
| 345          | Tableau    |


In [ ]:
SELECT candidate_id
FROM candidates
GROUP BY candidate_id
HAVING 'Python' = ANY(ARRAY_AGG(skill))
AND 'Tableau' = ANY(ARRAY_AGG(skill))
AND 'PostgreSQL' = ANY(ARRAY_AGG(skill))
ORDER BY 1

SELECT candidate_id
FROM candidates
WHERE skill IN ('Python', 'Tableau', 'PostgreSQL')
GROUP BY candidate_id
HAVING COUNT(*) = 3
ORDER BY candidate_id

Write a solution to find the second highest salary from the Employee table. If there is no second highest salary, return null (return None in Pandas).

In [ ]:
WITH rankings AS (
    SELECT salary, DENSE_RANK() OVER (ORDER BY salary DESC) rank_
    FROM Employee
)
SELECT COALESCE(
    (SELECT DISTINCT salary FROM rankings WHERE rank_ = 2), NULL) AS "SecondHighestSalary";


SELECT COALESCE(
    (
    SELECT DISTINCT salary
    FROM Employee
    ORDER BY salary DESC
    OFFSET 1 LIMIT 1),
    NULL
) AS "SecondHighestSalary"
FROM Employee
LIMIT 1


SELECT MAX(salary) AS "SecondHighestSalary"
FROM Employee
WHERE salary < (SELECT MAX(salary) FROM Employee)

In [ ]:
import pandas as pd

def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    
    salaries = employee['salary'].drop_duplicates()
    salaries.sort_values(ascending=False, inplace=True)
    return pd.DataFrame({'SecondHighestSalary' : [salaries.iloc[1] if salaries.shape[0] > 1 else None]})

Write a solution to find the nth highest salary from the Employee table. If there is no nth highest salary, return null.

Input: 
Employee table:

| id | salary |
|----|--------|
| 1  | 100    |
| 2  | 200    |
| 3  | 300    |

n = 2
Output: 

| getNthHighestSalary(2) |
|------------------------|
| 200                    |

Example 2:

Input: 
Employee table:

| id | salary |
|----|--------|
| 1  | 100    |

n = 2
Output: 

| getNthHighestSalary(2) |
|------------------------|
| null                   |

In [ ]:
CREATE OR REPLACE FUNCTION NthHighestSalary(N INT) RETURNS TABLE (Salary INT) AS $$
BEGIN
  RETURN QUERY (

    WITH rankings AS (
        SELECT e.salary, DENSE_RANK() OVER (ORDER BY e.salary DESC) rank_
        FROM Employee e
        ORDER BY e.salary DESC
    )

    SELECT COALESCE(
        (SELECT r.salary
        FROM rankings r
        WHERE rank_ = N
        LIMIT 1),
        NULL
    ) AS "NthHighestSalary"

  );
END;
$$ LANGUAGE plpgsql;

In [ ]:
import pandas as pd

def nth_highest_salary(employee: pd.DataFrame, N: int) -> pd.DataFrame:

    df = employee['salary'].drop_duplicates().sort_values(ignore_index=True, ascending=False)
    if N <= 0:
        return pd.DataFrame({f'getNthHighestSalary({N})':[None]})
    else:
        return pd.DataFrame({f'getNthHighestSalary({N})':[int(df.iloc[N-1]) if df.shape[0] >= N else None]})

if we wanted `df` to actually be a DataFrame when defined. In the previous snippet, it immediately became a series.

In [ ]:
import pandas as pd

def nth_highest_salary(employee: pd.DataFrame, N: int) -> pd.DataFrame:

    df = employee[['salary']].drop_duplicates().sort_values(by='salary', ignore_index=True, ascending=False)
    if N <= 0:
        return pd.DataFrame({f'getNthHighestSalary({N})':[None]})
    else:
        return pd.DataFrame({f'getNthHighestSalary({N})':[int(df.iloc[N-1]) if df.shape[0] >= N else None]})

Table: `Customers`

| Column Name | Type    |
|-------------|---------|
| id          | int     |
| name        | varchar |

id is the primary key (column with unique values) for this table.
Each row of this table indicates the ID and name of a customer.
 

Table: `Orders`

| Column Name | Type |
|-------------|------|
| id          | int  |
| customerId  | int  |

id is the primary key (column with unique values) for this table.
customerId is a foreign key (reference columns) of the ID from the Customers table.
Each row of this table indicates the ID of an order and the ID of the customer who ordered it.
 

Write a solution to find all customers who never order anything.

Return the result table in any order.

In [ ]:
SELECT name AS "Customers"
FROM Customers
WHERE id NOT IN (
    SELECT DISTINCT Orders.customerId
    FROM Orders
)

In [ ]:
def find_customers(customers: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    buying_customers = list(orders['customerId'].drop_duplicates())
    return pd.DataFrame({'Customers': [customers.loc[i, 'name'] for i in range(customers.shape[0]) if customers.loc[i, 'id'] not in buying_customers]})

def find_customers(customers: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    df = pd.merge(customers, orders, how='left', left_on='id', right_on='customerId')
    return df[df['customerId'].isna()][['name']].rename(columns={'name':'Customers'})

def find_customers(customers: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    return customers[~customers.id.isin(orders.customerId)][['name']].rename(columns={'name':'Customers'})

`reference`: https://datalemur.com/questions/time-spent-snaps?referralCode=NpkU8goA

This is the same question as problem #25 in the SQL Chapter of Ace the Data Science Interview!

Assume you're given tables with information on Snapchat users, including their ages and time spent sending and opening snaps.

Write a query to obtain a breakdown of the time spent sending vs. opening snaps as a percentage of total time spent on these activities grouped by age group. Round the percentage to 2 decimal places in the output.

**Notes:**

* Calculate the following percentages:
    * time spent sending / (Time spent sending + Time spent opening)
    * Time spent opening / (Time spent sending + Time spent opening)
* To avoid integer division in percentages, multiply by 100.0 and not 100.

In [ ]:
WITH agg_sum AS (
  SELECT age_bucket, activity_type, SUM(time_spent) sum_time, SUM(SUM(time_spent)) OVER(PARTITION BY age_bucket) act_total
  FROM activities JOIN age_breakdown USING (user_id)
  WHERE activity_type IN ('send', 'open')
  GROUP BY age_bucket, activity_type
)
SELECT *
FROM

(SELECT age_bucket, SUM(ROUND((sum_time / act_total)*100.0, 2)) send_perc
FROM agg_sum
WHERE activity_type = 'send'
GROUP BY age_bucket) send_

JOIN

(SELECT age_bucket, SUM(ROUND((sum_time / act_total)*100.0, 2)) open_perc
FROM agg_sum
WHERE activity_type = 'open'
GROUP BY age_bucket) open_

USING (age_bucket)
ORDER BY age_bucket

`reference`: https://leetcode.com/problems/game-play-analysis-i/

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Activity table:

| player_id | device_id | event_date | games_played |
|-----------|-----------|------------|--------------|
| 1         | 2         | 2016-03-01 | 5            |
| 1         | 2         | 2016-05-02 | 6            |
| 2         | 3         | 2017-06-25 | 1            |
| 3         | 1         | 2016-03-02 | 0            |
| 3         | 4         | 2018-07-03 | 5            |

Output: 

| player_id | first_login |
|-----------|-------------|
| 1         | 2016-03-01  |
| 2         | 2017-06-25  |
| 3         | 2016-03-02  |


In [ ]:
SELECT player_id, event_date AS first_login
FROM
    (SELECT player_id, event_date, ROW_NUMBER() OVER(PARTITION BY player_id ORDER BY event_date) row_
    FROM Activity
    ORDER BY player_id, event_date) act_rows
WHERE row_ = 1

In [ ]:
import pandas as pd

def game_analysis(activity: pd.DataFrame) -> pd.DataFrame:
    return activity[['player_id', 'event_date']].groupby(by='player_id', as_index=False).min().rename(columns={'event_date':'first_login'})

`reference`: https://leetcode.com/problems/duplicate-emails/description/

Write a solution to report all the duplicate emails. Note that it's guaranteed that the email field is not NULL.

Return the result table in any order.

The result format is in the following example.

Example 1:

Input: 
Person table:

| id | email   |
|----|---------|
| 1  | a@b.com |
| 2  | c@d.com |
| 3  | a@b.com |

Output: 

| Email   |
|---------|
| a@b.com |

Explanation: a@b.com is repeated two times.

In [ ]:
import pandas as pd
from collections import Counter

def duplicate_emails(person: pd.DataFrame) -> pd.DataFrame:
    count_dict = Counter(person['email'])
    return pd.DataFrame({'Email': list(set([e for e, c in count_dict.items() if c > 1]))})

def duplicate_emails(person: pd.DataFrame) -> pd.DataFrame:
    count_dict = {}
    for e in person['email']:
        if e not in count_dict:
            count_dict[e] = 1
        else:
            count_dict[e] += 1
    return pd.DataFrame({'Email': list(set([e for e, c in count_dict.items() if c > 1]))})

In [ ]:
SELECT DISTINCT email
FROM Person
GROUP BY email
HAVING count(email) > 1

Write a solution to find all dates' Id with higher temperatures compared to its previous dates (yesterday).

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Weather table:

| id | recordDate | temperature |
|----|------------|-------------|
| 1  | 2015-01-01 | 10          |
| 2  | 2015-01-02 | 25          |
| 3  | 2015-01-03 | 20          |
| 4  | 2015-01-04 | 30          |

Output: 

| id |
|----|
| 2  |
| 4  |

Explanation: 
In 2015-01-02, the temperature was higher than the previous day (10 -> 25).
In 2015-01-04, the temperature was higher than the previous day (20 -> 30).

In [ ]:
SELECT current.id AS "id"
FROM Weather current JOIN Weather previous ON current.recordDate = previous.recordDate + INTERVAL '1 DAY'
WHERE current.temperature > previous.temperature

* Reference: https://leetcode.com/problems/students-and-examinations/description/?envType=study-plan-v2&envId=top-sql-50
* Reference: https://stackoverflow.com/questions/3918570/what-is-the-difference-between-using-a-cross-join-and-putting-a-comma-between-th/31441463#31441463

Table: `Students`


| Column Name   | Type    |
|---------------|---------|
| student_id    | int     |
| student_name  | varchar |

student_id is the primary key (column with unique values) for this table.
Each row of this table contains the ID and the name of one student in the school.
 

Table: `Subjects`


| Column Name  | Type    |
|--------------|---------|
| subject_name | varchar |

subject_name is the primary key (column with unique values) for this table.
Each row of this table contains the name of one subject in the school.
 

Table: `Examinations`


| Column Name  | Type    |
|--------------|---------|
| student_id   | int     |
| subject_name | varchar |

There is no primary key (column with unique values) for this table. It may contain duplicates.
Each student from the Students table takes every course from the Subjects table.
Each row of this table indicates that a student with ID student_id attended the exam of subject_name.
 

Write a solution to find the number of times each student attended each exam.

Return the result table ordered by student_id and subject_name.

The result format is in the following example.

 

Example 1:

Input: 
Students table:

| student_id | student_name |
|------------|--------------|
| 1          | Alice        |
| 2          | Bob          |
| 13         | John         |
| 6          | Alex         |

Subjects table:

| subject_name |
|--------------|
| Math         |
| Physics      |
| Programming  |

Examinations table:

| student_id | subject_name |
|------------|--------------|
| 1          | Math         |
| 1          | Physics      |
| 1          | Programming  |
| 2          | Programming  |
| 1          | Physics      |
| 1          | Math         |
| 13         | Math         |
| 13         | Programming  |
| 13         | Physics      |
| 2          | Math         |
| 1          | Math         |

Output: 

| student_id | student_name | subject_name | attended_exams |
|------------|--------------|--------------|----------------|
| 1          | Alice        | Math         | 3              |
| 1          | Alice        | Physics      | 2              |
| 1          | Alice        | Programming  | 1              |
| 2          | Bob          | Math         | 1              |
| 2          | Bob          | Physics      | 0              |
| 2          | Bob          | Programming  | 1              |
| 6          | Alex         | Math         | 0              |
| 6          | Alex         | Physics      | 0              |
| 6          | Alex         | Programming  | 0              |
| 13         | John         | Math         | 1              |
| 13         | John         | Physics      | 1              |
| 13         | John         | Programming  | 1              |

Explanation: 
The result table should contain all students and all subjects.
Alice attended the Math exam 3 times, the Physics exam 2 times, and the Programming exam 1 time.
Bob attended the Math exam 1 time, the Programming exam 1 time, and did not attend the Physics exam.
Alex did not attend any exams.
John attended the Math exam 1 time, the Physics exam 1 time, and the Programming exam 1 time.

In [ ]:
SELECT all_.student_id, all_.student_name, all_.subject_name, COALESCE(attended_exams, 0) attended_exams
FROM
    (SELECT *
    FROM students, subjects) all_

FULL OUTER JOIN

    (SELECT student_id, student_name, subject_name, COUNT(*) attended_exams
    FROM Examinations JOIN Students USING (student_id)
    GROUP BY 1, 2, 3) exams_
ON all_.student_id = exams_.student_id AND all_.subject_name = exams_.subject_name
ORDER BY all_.student_id, all_.subject_name

In [ ]:
SELECT t.student_id, t.student_name, s.subject_name, COUNT(e.subject_name) attended_exams
FROM Students t CROSS JOIN Subjects s
LEFT JOIN Examinations e ON t.student_id = e.student_id AND s.subject_name = e.subject_name
GROUP BY t.student_id, t.student_name, s.subject_name
ORDER BY t.student_id, s.subject_name

* Reference: https://leetcode.com/problems/average-selling-price/?envType=study-plan-v2&envId=top-sql-50
* Acceptance Rate 39.1%

Write a solution to find the average selling price for each product. average_price should be rounded to 2 decimal places.

Return the result table in any order.

The result format is in the following example. 

Example 1:

Input: 
Prices table:

| product_id | start_date | end_date   | price  |
|------------|------------|------------|--------|
| 1          | 2019-02-17 | 2019-02-28 | 5      |
| 1          | 2019-03-01 | 2019-03-22 | 20     |
| 2          | 2019-02-01 | 2019-02-20 | 15     |
| 2          | 2019-02-21 | 2019-03-31 | 30     |

UnitsSold table:

| product_id | purchase_date | units |
|------------|---------------|-------|
| 1          | 2019-02-25    | 100   |
| 1          | 2019-03-01    | 15    |
| 2          | 2019-02-10    | 200   |
| 2          | 2019-03-22    | 30    |

Output: 

| product_id | average_price |
|------------|---------------|
| 1          | 6.96          |
| 2          | 16.96         |

Explanation: 
Average selling price = Total Price of Product / Number of products sold.
Average selling price for product 1 = ((100 * 5) + (15 * 20)) / 115 = 6.96
Average selling price for product 2 = ((200 * 15) + (30 * 30)) / 230 = 16.96

In [ ]:
SELECT p.product_id, COALESCE(ROUND(SUM(u.units * p.price)::NUMERIC / SUM(u.units), 2), 0) average_price
FROM UnitsSold u FULL OUTER JOIN Prices p ON (u.product_id = p.product_id) AND (u.purchase_date BETWEEN p.start_date AND p.end_date)
GROUP BY p.product_id

Joining on Prices should work.

In [ ]:
SELECT p.product_id, COALESCE(ROUND(SUM(u.units * p.price)::NUMERIC / SUM(u.units), 2), 0) average_price
FROM UnitsSold u RIGHT JOIN Prices p ON (u.product_id = p.product_id) AND (u.purchase_date BETWEEN p.start_date AND p.end_date)
GROUP BY p.product_id

Write a solution to find managers with at least five direct reports.

Return the result table in any order.

The result format is in the following example.

Example 1:

Input: 
Employee table:

| id  | name  | department | managerId |
|-----|-------|------------|-----------|
| 101 | John  | A          | null      |
| 102 | Dan   | A          | 101       |
| 103 | James | A          | 101       |
| 104 | Amy   | A          | 101       |
| 105 | Anne  | A          | 101       |
| 106 | Ron   | B          | 101       |

Output: 

| name |
|------|
| John |


In [ ]:
WITH atleastfive AS (
    SELECT m.id, m.name
    FROM Employee e JOIN Employee m ON e.managerId = m.id
    GROUP BY m.id, m.name
    HAVING COUNT(m.id) >= 5
)
SELECT name
FROM atleastfive

In [ ]:
WITH manager_info AS (
    SELECT m.id, m.name
    FROM Employee e JOIN Employee m ON e.managerId = m.id
)
SELECT name
FROM manager_info
GROUP BY id, name
HAVING COUNT(id) >= 5

Write a solution to find the percentage of the users registered in each contest rounded to two decimals.

Return the result table ordered by percentage in descending order. In case of a tie, order it by contest_id in ascending order.

The result format is in the following example.

Example 1:

Input: 
Users table:

| user_id | user_name |
|---------|-----------|
| 6       | Alice     |
| 2       | Bob       |
| 7       | Alex      |

Register table:

| contest_id | user_id |
|------------|---------|
| 215        | 6       |
| 209        | 2       |
| 208        | 2       |
| 210        | 6       |
| 208        | 6       |
| 209        | 7       |
| 209        | 6       |
| 215        | 7       |
| 208        | 7       |
| 210        | 2       |
| 207        | 2       |
| 210        | 7       |

Output: 

| contest_id | percentage |
|------------|------------|
| 208        | 100.0      |
| 209        | 100.0      |
| 210        | 100.0      |
| 215        | 66.67      |
| 207        | 33.33      |

Explanation: 
All the users registered in contests 208, 209, and 210. The percentage is 100% and we sort them in the answer table by contest_id in ascending order.
Alice and Alex registered in contest 215 and the percentage is ((2/3) * 100) = 66.67%
Bob registered in contest 207 and the percentage is ((1/3) * 100) = 33.33%

In [ ]:
-- how many users for each contest
-- divided by total number of users
SELECT contest_id, ROUND((COUNT(user_id)::DECIMAL / (SELECT COUNT(DISTINCT user_id) FROM Users)) * 100.0, 2) AS "percentage"
FROM Register
GROUP BY contest_id
ORDER BY percentage DESC, contest_id